# Preprocessing + Policy Learning Workflow

In this workflow, PyCFRL takes in an offline trajectory and then preprocesses the offline trajectory using `SequentialPreprocessor`. After that, the preprocessed trajectory is passed into `FQI` to train a counterfactually fair policy, which is the final output of the workflow. This workflow is appropriate if the user wants to train a policy using PyCFRL. The trained policy can be further evaluated on its value and counterfactual fairness, which is discussed in detail in the "Assessing Policies Using Real Data" notebook.

We begin by importing the libraries needed for this demonstration.

In [ ]:
import sys
sys.path.append("E:/learning/university/MiSIL/CFRL Python Package/CFRL")

In [1]:
pip install pycfrl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from pycfrl.reader import read_trajectory_from_dataframe, convert_trajectory_to_dataframe
from pycfrl.preprocessor import SequentialPreprocessor
from pycfrl.agents import FQI
np.random.seed(10) # ensure reproducibility
torch.manual_seed(10) # ensure reproducibility

## Data Loading

In this demonstration, we use an offline trajectory generated from a `SyntheticEnvironment` using some pre-specified transition rules. Although it is actually synthesized, we treat it as if it is from some unknown environment for pedagogical convenience in this demonstration.

The trajectory contains 500 individuals (i.e. $N=500$) and 10 transitions (i.e. $T=10$). The actions are binary ($0$ or $1$) and were sampled using a random policy that selects $0$ or $1$ randomly with equal probability. It is stored in a tabular format in a `.csv` file. The sensitive attribute variable is univariate, stored in the column `z1`. The legit values of the sensitive attribute are $0$ and $1$. The state variable is also univariate, stored in the column `state1`. The actions are stored in the column `action` and rewards in the column `reward`. The tabular data also includes an extra irrelevant column `timestamp`. 

We can load and view the tabular data.

In [3]:
trajectory = pd.read_csv('../data/sample_data_large_uni.csv')
trajectory

,Unnamed: 0,ID,timestamp,z1,action,reward,state1
0,0,1.0,1.0,0.0,NaN,NaN,1.324345
1,1,1.0,2.0,0.0,1.0,1.524345,-0.813722
2,2,1.0,3.0,0.0,1.0,-0.613722,-0.526683
3,3,1.0,4.0,0.0,1.0,-0.326683,-0.464447
4,4,1.0,5.0,0.0,1.0,-0.264447,-2.075518
...,...,...,...,...,...,...,...
5495,5495,500.0,7.0,1.0,1.0,-2.468460,-0.941954
5496,5496,500.0,8.0,1.0,1.0,-1.430345,-2.536595
5497,5497,500.0,9.0,1.0,0.0,-1.068298,-0.946557
5498,5498,500.0,10.0,1.0,0.0,-0.273278,-0.709017


We now read the trajectory from the tabular format into Trajectory Arrays.

In [4]:
zs, states, actions, rewards, ids = read_trajectory_from_dataframe(
                                                data=trajectory, 
                                                z_labels=['z1'], 
                                                state_labels=['state1'], 
                                                action_label='action', 
                                                reward_label='reward', 
                                                id_label='ID', 
                                                T=10
                                                )

## Preprocessor Training

Before preprocessing the trajectory, we need to first train a preprocessor. To mitigate overfitting, we use a random subset of 250 individuals in the trajectory to train the preprocessor. The remaining 250 individuals will be actually preprocessed. We now form these two sets.

In [5]:
(
    zs_train, zs_prepro, 
    states_train, states_prepro, 
    actions_train, actions_prepro, 
    rewards_train, rewards_prepro, 
    ids_train, ids_prepro
) = train_test_split(zs, states, actions, rewards, ids, test_size=0.5)

We now use the training set to train a `SequentialPreprocessor`.

In [6]:
sp = SequentialPreprocessor(z_space=[[0], [1]], 
                            num_actions=2, 
                            cross_folds=1, 
                            mode='single', 
                            reg_model='nn')
sp.train_preprocessor(zs=zs_train, xs=states_train, actions=actions_train, rewards=rewards_train)

100%|██████████| 1000/1000 [00:21<00:00, 45.76it/s]

The fluctuation in the loss is not small enough in at least one of the final 10 epochs during neural network training


(array([[[-0.32461696,  0.57096795],
         [ 0.22408737,  1.34364977],
         [-0.75577441,  1.3940598 ],
         ...,
         [-0.65848739, -0.12798007],
         [-0.64196535,  0.70769511],
         [-0.40729995,  1.56825744]],
 
        [[-0.2082495 ,  0.6873354 ],
         [ 1.26062998,  2.39690117],
         [-0.63231817,  1.84835638],
         ...,
         [-0.81515169,  0.43440456],
         [-1.39260415,  0.45079704],
         [-2.79777832, -1.66740244]],
 
        [[ 1.20425904,  2.09984394],
         [-0.71320386,  1.66238533],
         [ 0.96579847,  3.91839007],
         ...,
         [ 0.53464998,  2.41311652],
         [-1.10176242,  0.37004053],
         [ 0.5821277 ,  2.51716889]],
 
        ...,
 
        [[ 1.26530747,  2.16089238],
         [-2.15263285,  0.22994996],
         [-0.98369375,  1.14478895],
         ...,
         [-2.50899961, -1.07172801],
         [-0.83936513,  0.45569243],
         [-0.8893273 ,  0.26598686]],
 
        [[ 0.93616403,  1.831

## Policy Learning

We now train a counterfactually fair policy using the preprocessor `sp` and the set of trajectory data that is not used to train `sp`. We begin by initializing an `FQI` agent with `sp` as its internal preprocessor.

In [7]:
agent = FQI(num_actions=2, model_type='nn', preprocessor=sp)

We now perform training. Since we set `preprocess=True` in `train()`, `agent` will use its internal preprocessor (i.e. `sp`) to automatically preprocess the input training trajectory before using the trajectory for policy learning. Therefore, we can directly pass in the unpreprocessed `states_prepro` and `rewards_prepro`. 

In [8]:
agent.train(zs=zs_prepro, 
            xs=states_prepro, 
            actions=actions_prepro, 
            rewards=rewards_prepro, 
            max_iter=100, 
            preprocess=True)

  2%|▏         | 2/100 [00:01<01:01,  1.59it/s]
The fluctuation in the loss is not small enough in at least one of the final 10 epochs during neural network training
100%|██████████| 100/100 [01:03<00:00,  1.58it/s]

The fluctuation in the Q values is not small enough in at least one of the final 5 iterations during FQI training


## Decision-making Using the Trained Policy

Now we have trained a counterfactually fair policy. We can use its `act()` method to make decisions. At actual deployment, the single-step trajectory passed into `act()` is often generated from the real enviornment at the time. However, in this part, we reuse the training trajectory data for convenience.

We first take actions for the initial time step (i.e. $a_0$). Again, we only pass in unpreprocessed states and rewards because they will be automatically preprocessed inside `act()` before being used for decision-making. We set `xtm1` and `atm1` to `None`. This tells `agent` that we are taking the initial action and that the buffer of the internal preprocessor should be reset.

In [9]:
a0 = agent.act(z=zs, 
               xt=states[:, 0], 
               xtm1=None, 
               atm1=None, 
               preprocess=True)
a0

array([1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,

We now take the second action. We still pass in unpreprocessed states and rewards, and we specify non-`None` `xtm1` and `atm1` to tell `agent` not to reset the buffer of its internal preprocessor.

In [10]:
a1 = agent.act(z=zs, 
               xt=states[:, 1], 
               xtm1=states[:, 0], 
               atm1=a0, 
               preprocess=True)
a1

array([1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,

## Alternative: Using All Individuals for Policy Learning

Sometimes, the number of individuals in the trajectory is small. In this case, if we only use a subset of individuals for policy learning, the amount of training trajectory data might be too small to be useful for policy learning. Fortunately, when we set `cross_folds` to a relatively large number, we can directly preprocess all individuals using the `train_preprocessor()` function, and the preprocessed trajectory can be used for policy learning.

When `cross_folds=K` where `K` is greater than 1, `train_preprocessor()` will internally divide the training data into `K` folds. For each $i=1,\dots,K$, it trains a transition dynamics model based on all the folds other than the $i$-th one, and this model is then used to preprocess data in the $i$-th fold. This results in `K` folds of preprocessed data, each of which is processed using a model that is trained on the other folds. These `K` folds of preprocessed data are then combined and returned by `train_preprocessor()`. This method allows us to preprocess all individuals in the trajectory while reducing overfitting.

To use this functionality, we first initialize a `SequentialPreprocessor` with `cross_folds` greater than 1. We use `cross_folds=5` here.

In [11]:
sp_cf5 = SequentialPreprocessor(z_space=[[0], [1]], 
                                num_actions=2, 
                                cross_folds=5, 
                                mode='single', 
                                reg_model='nn')

We now simultaneously train the preprocessor and preprocess all individuals in the trajectory using the precedure described above.

In [12]:
states_tilde_cf5, rewards_tilde_cf5 = sp_cf5.train_preprocessor(zs=zs, 
                                                                xs=states, 
                                                                actions=actions, 
                                                                rewards=rewards)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:39<00:00, 25.38it/s]


We now train a policy using `FQI` and the preprocessed data. Note that the input training data `state_tilde` and `rewards_tilde` are already preprocessed. Thus, we set `preprocess=False` during training so that the input trajectory will not be preprocessed again by the internal preprocessor (i.e. `sp_cf5`).

In [13]:
agent_cf5 = FQI(num_actions=2, model_type='nn', preprocessor=sp_cf5)
agent_cf5.train(zs=zs, 
                xs=states_tilde_cf5, 
                actions=actions, 
                rewards=rewards_tilde_cf5, 
                max_iter=100, 
                preprocess=False)

 13%|█▎        | 13/100 [00:12<01:43,  1.20s/it]
The fluctuation in the loss is not small enough in at least one of the final 10 epochs during neural network training
100%|██████████| 100/100 [01:40<00:00,  1.01s/it]

The fluctuation in the Q values is not small enough in at least one of the final 5 iterations during FQI training


We now take the initial action using `agent_cf5`.

In [14]:
a0 = agent_cf5.act(z=zs, 
                   xt=states[:, 0], 
                   xtm1=None, 
                   atm1=None, 
                   preprocess=True)
a0

array([1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,

We now take the second action using `agent_cf5`.

In [15]:
a1 = agent_cf5.act(z=zs, 
                   xt=states[:, 1], 
                   xtm1=states[:, 0], 
                   atm1=a0, 
                   preprocess=True)
a1

array([1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,